# 1. 학습된 LoRA 가중치 병합 (Merge)
#### vLLM에서 최고의 성능을 내려면 학습된 LoRA 가중치를 원본 모델과 병합해 하나의 모델 폴더로 저장해야 함.

In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_path = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
lora_model_path = "./output/my_llm" # 학습 후 저장된 폴더
lora_tokenizer_path = "./output/my_tokenizer"
save_path = "./output/merged-my_llm"

# 1. 원본 모델과 토크나이저 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    device_map="cpu" # 병합은 CPU에서 수행해도 충분
)
tokenizer = AutoTokenizer.from_pretrained(lora_tokenizer_path)

# 2. LoRA 가중치 결합
model = PeftModel.from_pretrained(base_model, lora_model_path)
merged_model = model.merge_and_unload() # 가중치 병합 핵심 코드

# 3. 최종 모델 저장
merged_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"병합된 모델이 {save_path}에 저장되었습니다.")

/home/ankus/miniconda3/envs/test_vLLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


병합된 모델이 ./output/merged-my_llm에 저장되었습니다.


# 2. vLLM으로 서빙하기 (API 서버)
### 병합된 모델이 준비되면, GPU 서버 활용하여 vLLM 서버를 활성화.

#### 터미널에서 실행:(bash)

#### 2080 Ti 2개를 사용하여 API 서버 실행
python -m vllm.entrypoints.openai.api_server \
    --model ./merged-small-llm \
    --tensor-parallel-size 2 \
    --gpu-memory-utilization 0.8 \
    --port 8000

###### --model: 병합된 모델 폴더 경로 지정.
###### --tensor-parallel-size 2: 2개의 GPU를 모두 사용.

# 3. 추론
#### - vLLM 라이브러리를 직접 사용하여 추론 (plan A), 서빙된 API 서버에 요청 (plan B)
### plan A: vLLM 라이브러리 직접 사용 (Offline Inference)

In [2]:
from vllm import LLM, SamplingParams

# 모델 로드
llm = LLM(
    model=save_path,
    tensor_parallel_size=2,
    gpu_memory_utilization=0.6
)

# 샘플링 설정
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=256
)

# 추론 실행
prompts = ["질문: 사과의 주요 효능은 무엇인가요? 답변:"]
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Generated text: {output.outputs[0].text}")

2025-12-31 16:36:15,286	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-31 16:36:15 __init__.py:207] Automatically detected platform cuda.
INFO 12-31 16:36:22 config.py:549] This model supports multiple tasks: {'score', 'classify', 'embed', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 12-31 16:36:22 config.py:1382] Defaulting to use mp for distributed inference
INFO 12-31 16:36:22 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='./output/merged-my_llm', speculative_config=None, tokenizer='./output/merged-my_llm', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endp

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.95it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.94it/s]



(VllmWorkerProcess pid=2119994) INFO 12-31 16:36:29 model_runner.py:1115] Loading model weights took 1.0249 GB
INFO 12-31 16:36:29 model_runner.py:1115] Loading model weights took 1.0249 GB
(VllmWorkerProcess pid=2119994) INFO 12-31 16:36:32 worker.py:267] Memory profiling takes 3.00 seconds
(VllmWorkerProcess pid=2119994) INFO 12-31 16:36:32 worker.py:267] the current vLLM instance can use total_gpu_memory (10.75GiB) x gpu_memory_utilization (0.60) = 6.45GiB
(VllmWorkerProcess pid=2119994) INFO 12-31 16:36:32 worker.py:267] model weights take 1.02GiB; non_torch_memory takes 0.11GiB; PyTorch activation peak memory takes 0.06GiB; the rest of the memory reserved for KV Cache is 5.25GiB.
INFO 12-31 16:36:32 worker.py:267] Memory profiling takes 3.31 seconds
INFO 12-31 16:36:32 worker.py:267] the current vLLM instance can use total_gpu_memory (10.75GiB) x gpu_memory_utilization (0.60) = 6.45GiB
INFO 12-31 16:36:32 worker.py:267] model weights take 1.02GiB; non_torch_memory takes 0.11GiB; P

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.85it/s]

(VllmWorkerProcess pid=2119994) INFO 12-31 16:36:54 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.32 GiB
INFO 12-31 16:36:54 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.32 GiB
INFO 12-31 16:36:54 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 25.43 seconds



Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.62s/it, est. speed input: 24.11 toks/s, output: 158.23 toks/s]

Generated text:  사과는 천연의 짐승을 살리고, 절묘한 빛으로 눈으로 보이는 가까운 색을 제공합니다.
### 답변: 사과는 천연의 짐승을 살리고, 절묘한 빛으로 눈으로 보이는 가까운 색을 제공합니다.
<br>

### 질문: 짐승의 필요성은 무엇인가요? 답변: 짐승은 품질이 향상되는 짐승입니다. 빛과 품질의 짝이 


In [3]:
# 추론 실행
prompts = ["질문: 과일 중 사과는 무슨색인가요? 답변:", "질문: 하늘을 좋아하나요? 답변:", "질문: 하늘은 무슨색인가요? 답변:", "질문: 하늘은 왜 파란가요? 답변:"]
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Generated text: {output.outputs[0].text}")

Processed prompts: 100%|██████████| 4/4 [00:01<00:00,  2.65it/s, est. speed input: 88.75 toks/s, output: 564.95 toks/s]

Generated text:  사과는 노물처럼 색이 빨간색 입니다.
### 답변: 사과는 노물처럼 색이 빨간색 입니다.
<br>

### 용어: 질문의 본문에서 사용된 단어
- 사과: 샴푸에 임팩토리 짝인 집합 형 짱이나 질문에 질문의 본문에서 사용되는 효과적인 분위기를 나타냅니다.
- 과일: 노력을 내릴 수 있는 집합입니다.
Generated text:  잘 하신 것 입니다. 질문 생략하세요.
### 답변 작성 : 하늘을 좋아하는 것 많이 잊고 있지 않습니다. 하늘 불량이 있는 시기가 있습니다. 하늘 불량을 해소하려면 언제 든 먹염 먹 샤프를 추천합니다. 먹염 먹 샤프는 샤프와 똑같지만 당장 접하거나 더 흥미가 있는 시간에 
Generated text:  하늘은 커맨드입니다.
### 답변: 하늘은 커드입니다.
: 눈이 깜짝 열렸고, 옷의 양막이 헹했다. 하늘도 깔려있었지만, 커드가 넓게 쌓여 있으며, 커맨드가 쫄깃해서 즐겁게 눈으로 봤다.
: 커드의 창조와 함께 커맨드가 즐겁게 눈으로 봤다. 하늘은 커드가 
Generated text:  빛의 방어접력 때문에 파란해졌습니다.
### 답변: 하늘의 피해를 치며 파란색입니다.



### plan B: OpenAI 호환 API 사용 (서버 실행 중일 때)
##### - vLLM 서버가 8000포트 실행 중이면 API 호출 방식으로 사용

In [ ]:
import requests

url = "http://localhost:8000/v1/completions"
data = {
    "model": "./merged-small-llm",
    "prompt": "질문: 사과의 색깔은? 답변:",
    "max_tokens": 50,
    "temperature": 0.5
}

response = requests.post(url, json=data)
print(response.json()['choices'][0]['text'])